In [1]:
import numpy as np, pandas as pd
from scipy.optimize import minimize
from tqdm import tqdm

In [3]:
# Tversky, A., & Kahneman, D. (1992). Advances in prospect theory: Cumulative representation of uncertainty. Journal of Risk and uncertainty, 5, 297-323. https://link.springer.com/article/10.1007/BF00122574
# Charpentier, C. J., Aylward, J., Roiser, J. P., & Robinson, O. J. (2017). Enhanced risk aversion, but not loss aversion, in unmedicated pathological anxiety. Biological psychiatry, 81(12), 1014-1022. https://www.sciencedirect.com/science/article/pii/S0006322316331110?via%3Dihub#bib28

In [4]:
subj_df = pd.read_csv('example_data\\MS017_task_data.csv')

In [5]:
def negll_prospect3(params, subj_df):
    risk_aversion, loss_aversion, inverse_temp = params

    # init list of choice prob predictions
    choiceprob_list = []


    #loop through trials
    for trial in range(len(subj_df)):

        # get relevant trial info
        trial_info = subj_df.iloc[trial]
        high_bet = trial_info['High.Bet']
        low_bet = trial_info['Low.Bet']
        safe_bet = trial_info['Safe.Bet']
        trial_type = trial_info['TrialType']
        choice = trial_info['Gamble.Choice']
        outcome = trial_info['Profit']

        # transform to high bet value to utility (gamble)
        if high_bet >= 0:
            weighted_high_bet = 0.5 * (high_bet)**risk_aversion
        else:
            weighted_high_bet = -0.5 * loss_aversion * (-high_bet)**risk_aversion
        
        # transform to low bet value to utility (gamble)
        if low_bet >= 0:
            weighted_low_bet = 0.5 * (low_bet)**risk_aversion
        else:
            weighted_low_bet = -0.5 * loss_aversion * (-low_bet)**risk_aversion
        
        util_gamble = weighted_high_bet + weighted_low_bet

        # transform safe bet value to utility (safe)
        if safe_bet >= 0:
            util_safe = (safe_bet)**risk_aversion
        else:
            util_safe = -loss_aversion * (-safe_bet)**risk_aversion

        #print(f'{trial_type}: U(gamble) = {util_gamble:.3f}; U(safe) = {util_safe:.3f}')

        # convert EV to choice probabilities via softmax
        p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
        p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) + np.exp(inverse_temp*util_safe) )
        
        #print(f'{trial_type}: P(gamble) = {p_gamble:.3f}', f'P(safe) = {p_safe:.3f}')

        # append probability of chosen options
        if choice == 'gamble':
            choiceprob_list.append(p_gamble)
        elif choice == 'safe':
            choiceprob_list.append(p_safe)

    # compute the neg LL of choice probabilities across the entire task
    negLL = -np.sum(np.log(choiceprob_list))
    
    if np.isnan(negLL):
        return np.inf
    else:
        return negLL

In [6]:
# gradient descent to minimize neg LL
res_nll = np.inf # set initial neg LL to be inf

# rho    = risk_aversion_
# lambda = loss_aversion
# beta   = inverse_temp

# guess several different starting points for rho
for rho_guess in tqdm(np.linspace(0,6,4)):
        for lambda_guess in np.linspace(0,20,4):
            for beta_guess in np.linspace(1,20,4):
        
                # guesses for alpha, theta will change on each loop
                init_guess = (rho_guess, lambda_guess, beta_guess)
                
                # minimize neg LL
                result = minimize(negll_prospect3, 
                                 init_guess, 
                                 (subj_df), 
                                 bounds=((0,6),(0,50),(.001,50)))
                
                # if current negLL is smaller than the last negLL,
                # then store current data
                if result.fun < res_nll:
                    res_nll = result.fun
                    param_fits = result.x
                    risk_aversion, loss_aversion, inverse_temp = param_fits

# also, compute BIC
# note: we don't need the -1 because 
# we already have the negative log likelihood!
BIC = len(init_guess) * np.log(len(subj_df)) + 2*res_nll

print(fr'risk_aversion = {risk_aversion:.4f}, loss_aversion = {loss_aversion:.4f}, inverse_temp = {inverse_temp:.4f}')
print(fr'BIC = {BIC:.3f}')

  0%|          | 0/4 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in exp
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in exp
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
100%|██████████| 4/4 [01:42<00:00, 25.62s/it]

risk_aversion = 1.3260, loss_aversion = 0.9892, inverse_temp = 6.3368
BIC = 148.850


<hr>

In [9]:
def negll_prospect4(params, subj_df):
    risk_aversion_pos, risk_aversion_neg, loss_aversion, inverse_temp = params

    # init list of choice prob predictions
    choiceprob_list = []

    #loop through trials
    for trial in range(len(subj_df)):

        # get relevant trial info
        trial_info = subj_df.iloc[trial]
        high_bet = trial_info['High.Bet']
        low_bet = trial_info['Low.Bet']
        safe_bet = trial_info['Safe.Bet']
        trial_type = trial_info['TrialType']
        choice = trial_info['Gamble.Choice']
        outcome = trial_info['Profit']

        # transform to high bet value to utility (gamble)
        if high_bet >= 0:
            weighted_high_bet = 0.5 * (high_bet)**risk_aversion_pos
        else:
            weighted_high_bet = -0.5 * loss_aversion * (-high_bet)**risk_aversion_neg
        
        # transform to low bet value to utility (gamble)
        if low_bet >= 0:
            weighted_low_bet = 0.5 * (low_bet)**risk_aversion_pos
        else:
            weighted_low_bet = -0.5 * loss_aversion * (-low_bet)**risk_aversion_neg
        
        util_gamble = weighted_high_bet + weighted_low_bet

        # transform safe bet value to utility (safe)
        if safe_bet >= 0:
            util_safe = (safe_bet)**risk_aversion_pos
        else:
            util_safe = -loss_aversion * (-safe_bet)**risk_aversion_neg

        # convert EV to choice probabilities via softmax
        p_gamble = np.exp(inverse_temp*util_gamble) / ( np.exp(inverse_temp*util_gamble) +  np.exp(inverse_temp*util_safe) )
        p_safe = np.exp(inverse_temp*util_safe) / ( np.exp(inverse_temp*util_gamble) +  np.exp(inverse_temp*util_safe) )
        
        #print(trial_type, f'{p_gamble:.3f}', f'{p_safe:.3f}')

        # append probability of chosen options
        if choice == 'gamble':
            choiceprob_list.append(p_gamble)
        elif choice == 'safe':
            choiceprob_list.append(p_safe)

    # compute the neg LL of choice probabilities across the entire task
    negLL = -np.sum(np.log(choiceprob_list))

    return negLL

In [10]:
# gradient descent to minimize neg LL
res_nll = np.inf # set initial neg LL to be inf

# rho_pos = risk_aversion_pos
# rho_neg = risk_aversion_neg
# lambda  = loss_aversion
# beta    = inverse_temp

# guess several different starting points
for rho_pos_guess in tqdm(np.linspace(0,6,4)):
    for rho_neg_guess in np.linspace(0,6,4):
        for lambda_guess in np.linspace(0,20,4):
            for beta_guess in np.linspace(1,20,4):
        
                # guesses for alpha, theta will change on each loop
                init_guess = (rho_pos_guess, rho_neg_guess, lambda_guess, beta_guess)
                
                # minimize neg LL
                result = minimize(negll_prospect4, 
                                 init_guess, 
                                 (subj_df), 
                                 bounds=((0,6),(0,6),(0,50),(.001,50)))
                
                # if current negLL is smaller than the last negLL,
                # then store current data
                if result.fun < res_nll:
                    res_nll = result.fun
                    param_fits = result.x
                    risk_aversion_pos, risk_aversion_neg, loss_aversion, inverse_temp = param_fits

# also, compute BIC
# note: we don't need the -1 because 
# we already have the negative log likelihood!
BIC = len(init_guess) * np.log(len(subj_df)) + 2*res_nll

print(fr'risk_aversion_pos = {risk_aversion_pos:.4f}, risk_aversion_neg = {risk_aversion_neg:.4f}, loss_aversion = {loss_aversion:.4f}, inverse_temp = {inverse_temp:.4f}')
print(fr'BIC = {BIC:.3f}')

  0%|          | 0/4 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in exp
c:\ProgramData\Anaconda3\envs\ccp\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in log
100%|██████████| 4/4 [11:31<00:00, 172.81s/it]

risk_aversion_pos = 1.0509, risk_aversion_neg = 1.4683, loss_aversion = 0.8937, inverse_temp = 8.1392
BIC = 149.926
